In [1]:
# Import matplotlib.pyplot
from random import random

import matplotlib.pyplot as plt
# Import pandas
import pandas as pd
# Import geopandas
import geopandas as gpd
# import Folium
import folium
import shapely.geometry

In [6]:
# Vamos por nuestros datos
df = pd.read_csv('DATASET/recorrido_1.csv')
# Vemos su contenido
df.head()

,Coordenadas,Tipo Transporte,Coordenadas de Accidente,Tipo Accidente,Tiempo,Costo,Descripcion Accidente
0,"(19.383082625228962, -99.1906271466764)",CAMINAR,Ninguna,Ninguno,05:30,$0.00,NaN
1,"(19.38367013560702, -99.19071438138118)",CAMINAR,Ninguna,Ninguno,05:33,$0.00,NaN
2,"(19.383189401820953, -99.18691637376372)",CAMINAR,Ninguna,Ninguno,05:37,$0.00,NaN
3,"(19.383189401820953, -99.18691637376372)",CAMION,Ninguna,Ninguno,05:42,$7.00,NaN
4,"(19.362798100345877, -99.18939848852227)",CAMION,Ninguna,Ninguno,05:50,$0.00,NaN


In [7]:
# Vemos su descripcion
df.info()

# Vemos su tamaño mxn , m = filas, n = columnas
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Coordenadas               85 non-null     object
 1   Tipo Transporte           85 non-null     object
 2   Coordenadas de Accidente  85 non-null     object
 3   Tipo Accidente            85 non-null     object
 4   Tiempo                    85 non-null     object
 5   Costo                     85 non-null     object
 6   Descripcion Accidente     3 non-null      object
dtypes: object(7)
memory usage: 4.8+ KB


(85, 7)

In [8]:
# Acomodamos los datos
df.Coordenadas =  df.Coordenadas.str.strip('()')
df[['Latitude', 'Longitude']] = df.Coordenadas.str.split(',', expand=True)
df.Costo =  df.Costo.str.strip('$')
# Casteamos los datos al tipo de dato
df = df.astype({'Longitude': float, 'Latitude': float, 'Costo': float})

In [9]:
# Eliminamos la columna Coordenadas
df = df.drop('Coordenadas', axis=1)
df = df.drop('Coordenadas de Accidente', axis=1)
df = df.drop('Tipo Accidente', axis=1)
df = df.drop('Descripcion Accidente', axis=1)
# El resultado
df.head()

,Tipo Transporte,Tiempo,Costo,Latitude,Longitude
0,CAMINAR,05:30,0.0,19.383083,-99.190627
1,CAMINAR,05:33,0.0,19.383670,-99.190714
2,CAMINAR,05:37,0.0,19.383189,-99.186916
3,CAMION,05:42,7.0,19.383189,-99.186916
4,CAMION,05:50,0.0,19.362798,-99.189398


In [11]:
df.info()

df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tipo Transporte  85 non-null     object 
 1   Tiempo           85 non-null     object 
 2   Costo            85 non-null     float64
 3   Latitude         85 non-null     float64
 4   Longitude        85 non-null     float64
dtypes: float64(3), object(2)
memory usage: 3.4+ KB


(85, 5)

In [36]:
# Pasamos la informacion a un GeoDataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs ="EPSG:4326")

gdf['loc'] = gdf.geometry.apply(lambda x:shapely.geometry.mapping(x))
# Show
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Tipo Transporte  85 non-null     object  
 1   Tiempo           85 non-null     object  
 2   Costo            85 non-null     float64 
 3   Latitude         85 non-null     float64 
 4   Longitude        85 non-null     float64 
 5   geometry         85 non-null     geometry
 6   loc              85 non-null     object  
dtypes: float64(3), geometry(1), object(3)
memory usage: 4.8+ KB


In [37]:
gdf.head()

,Tipo Transporte,Tiempo,Costo,Latitude,Longitude,geometry,loc
0,CAMINAR,05:30,0.0,19.383083,-99.190627,POINT (-99.19063 19.38308),"{'type': 'Point', 'coordinates': (-99.19062714..."
1,CAMINAR,05:33,0.0,19.383670,-99.190714,POINT (-99.19071 19.38367),"{'type': 'Point', 'coordinates': (-99.19071438..."
2,CAMINAR,05:37,0.0,19.383189,-99.186916,POINT (-99.18692 19.38319),"{'type': 'Point', 'coordinates': (-99.18691637..."
3,CAMION,05:42,7.0,19.383189,-99.186916,POINT (-99.18692 19.38319),"{'type': 'Point', 'coordinates': (-99.18691637..."
4,CAMION,05:50,0.0,19.362798,-99.189398,POINT (-99.18940 19.36280),"{'type': 'Point', 'coordinates': (-99.18939848..."


In [39]:
cdmx = [19.432608, -99.133209]

# Creamos el mapa
m = folium.Map(location=cdmx, zoom_start=12, tiles='cartodbpositron')

In [40]:
geopath = gdf.geometry.to_json()
#print(geopath)
poligonos = folium.features.GeoJson(geopath)
m.add_child(poligonos)
folium.LayerControl().add_to(m)
m

In [31]:
geopath = gdf.loc #geometry.to_json()
geo_df_list = [[point.xy[0][0], point.xy[1][0]] for point in gdf.geometry ]

geo_df_list
for coordinates in geo_df_list:
    m.add_child(folium.Marker(location = coordinates, icon = folium.Icon(color = "green", icon='fa-person-walking')))

m

In [32]:
points = gdf[['Longitude', 'Latitude']].values.tolist()

In [33]:
from numpy import sin, cos, arccos, pi, round

x = points[0]
y = points[1]

def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

def getDistanceBetweenPointsNew(latitude1, longitude1, latitude2, longitude2, unit = 'miles'):

    theta = longitude1 - longitude2

    distance = 60 * 1.1515 * rad2deg(
        arccos(
            (sin(deg2rad(latitude1)) * sin(deg2rad(latitude2))) +
            (cos(deg2rad(latitude1)) * cos(deg2rad(latitude2)) * cos(deg2rad(theta)))
        )
    )

    if unit == 'miles':
        return round(distance, 2)
    if unit == 'kilometers':
        return round(distance * 1.609344, 3)


In [34]:
distances = []
for i in range(0, len(points)):
    k = i + 1
    if k == len(points):
        print("Se acabo")
    else:
        x = points[i]
        x2 = points[k]
        distances.append(getDistanceBetweenPointsNew(x[0], x[1], x2[0], x2[1], 'kilometers') * 1000)
        # print("[{0}] vs [{1}]".format(points[i], points[k]))

distances
df_distances = pd.DataFrame(distances, columns=['Distancias'])
df_distances

Se acabo


,Distancias
0,14.0
1,422.0
2,0.0
3,455.0
4,15.0
...,...
79,6.0
80,37.0
81,59.0
82,16.0


In [42]:
m = gdf.explore(tiles="CartoDB positron")
folium.PolyLine(gdf[['Latitude','Longitude']].values.tolist(), color="green", weight=2.5, opacity=1).add_to(m)
m